<a href="https://colab.research.google.com/github/JT-3710A/Neural-Network--Workshop/blob/main/notebook/cnn_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision

In [ ]:
from torchvision.transforms import transforms

transform = transforms.Compose([transforms.ToTensor()])

In [ ]:
train_set = torchvision.datasets.MNIST(root = "data",
                                              train = True,
                                              download=True,
                                              transform = transform)

In [ ]:
test_set = torchvision.datasets.MNIST(root = "data",
                                              train = False,
                                              download=True,
                                             transform = transform)

In [ ]:
train_set

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_loader = DataLoader(train_set, batch_size = 32, shuffle=True)
test_loader = DataLoader(test_set, batch_size = 32, shuffle=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)), cmap='gray')
    plt.show()

images, labels = next(iter(train_loader))

plt.figure(figsize=(10, 4))
imshow(torchvision.utils.make_grid(images, nrow=16))

In [ ]:
import torch.nn as nn

In [ ]:
class BroadwayModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.cnn1=nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2)
    self.cnn2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2)
    self.cnn3=nn.Conv2d(in_channels=32,out_channels=64,kernel_size=5,stride=1,padding=2) # Changed out_channels to 64
    self.pooling = nn.MaxPool2d(kernel_size=2, stride=2) # Define pooling layer
    self.fc_1 = nn.Linear(576, 512) # 64 channels * 3x3 feature map size = 576
    self.fc_2 = nn.Linear(512, 256)
    self.fc_3 = nn.Linear(256, 10)
    self.relu = nn.ReLU()
    self.flatten = nn.Flatten()

  def forward(self, x):
    x=self.relu(self.cnn1(x))
    x=self.pooling(x)
    x=self.relu(self.cnn2(x))
    x=self.pooling(x)
    x=self.relu(self.cnn3(x))
    x=self.pooling(x) # Add the third pooling layer
    x = self.flatten(x)
    # print("shape of x isss")
    # print(x.shape)
    x = self.relu(self.fc_1(x))
    x = self.relu(self.fc_2(x))
    x = self.fc_3(x)
    return x

In [ ]:
model = BroadwayModel()

In [ ]:
!pip install torchinfo -q

In [ ]:
# from torchinfo import summary

# batch_size = 64
# summary(model, input_size=(batch_size, 1, 28, 28))

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
100
# 10

In [ ]:
model.train()

for i in range(15):
  train_loss = 0
  for images, labels in train_loader:
    # images = images.to('cuda')
    # labels = labels.to('cuda')

    outputs = model(images)
    loss = criterion(outputs, labels)
    train_loss +=loss
    optimizer.zero_grad()  # clear old gradient
    loss.backward()   # gradient calculation using chain rule
    optimizer.step()  # update weights wm = w0 - n *gradient(loss.backward())

  final_loss = train_loss / len(train_loader)
  print('my loss is', final_loss)

In [ ]:
torch.save(model.state_dict(), 'my_first_model.pth')

In [ ]:
## Inferencing

In [ ]:
model = BroadwayModel()

In [ ]:
model_weight = torch.load('/content/my_first_model.pth', map_location = 'cuda')

model.load_state_dict(model_weight)

In [ ]:
model.eval()
test_loss = 0
correct = 0
total = 0

for images, label in test_loader:
  with torch.no_grad():
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += label.size(0)
    correct += (predicted == label).sum().item()

    loss = criterion(outputs, label)
    test_loss +=loss

final_test_loss = test_loss / len(test_loader)

accuracy = 100 * (correct / total )
print('my accuracy is ', accuracy)
print('test loss is', final_test_loss)